Functions
Python and C++ use fundamentally different ways of managing the memory and lifetime of objects managed by them. 
This can lead to issues when creating bindings for functions that return a non-trivial type. 
Just by looking at the type information, it is not clear whether Python should take charge of the returned value and eventually free its resources, or if this is handled on the C++ side. 
For this reason, pybind11 provides a several return value policy annotations that can be passed to the module_::def() and class_::def() functions. The default policy is return_value_policy::automatic

In [ ]:
/* Function declaration */
Data *get_data() { return _data; /* (pointer to a static data structure) */ }

/* Binding code */
m.def("get_data", &get_data); // <-- will cause crash when called from Python

When get_data() is called from Python, the return value (a native C++ static data structure) must be wrapped to turn it into a usable Python type. 
In this case, the default return value policy (return_value_policy::automatic) causes pybind11 to assume ownership of the static _data instance.

When Python’s garbage collector eventually deletes the Python wrapper, pybind11 will also attempt to delete the C++ instance (via operator delete()) due to the implied ownership. 
At this point, the entire application will come crashing down, though errors could also be more subtle and involve silent data corruption.

In the above example, the policy return_value_policy::reference should have been specified so that the global data instance is only referenced without any implied transfer of ownership, i.e.:

In [ ]:
m.def("get_data", &get_data, py::return_value_policy::reference);

On the other hand, this is not the right policy for many other situations, where ignoring ownership could lead to resource leaks.
As a developer using pybind11, it’s important to be familiar with the different return value policies, including which situation calls for which one of them. The following table provides an overview of available policies:

In [ ]:
/* Reference an existing object (i.e. do not create a new copy) and take ownership. 
    Python will call the destructor and delete operator when the object’s reference count reaches zero. 
    Undefined behavior ensues when the C++ side does the same, or when the data was not dynamically allocated. 
*/
m.def("get_data", &get_data, return_value_policy::take_ownership);

In [ ]:
/*
    Create a new copy of the returned object, which will be owned by Python. 
    This policy is comparably safe because the lifetimes of the two instances are decoupled.
*/
m.def("get_data", &get_data, return_value_policy::copy);

In [ ]:
/*
    Use std::move to move the return value contents into a new instance that will be owned by Python. 
    This policy is comparably safe because the lifetimes of the two instances (move source and destination) are decoupled.
*/
m.def("get_data", &get_data,return_value_policy::move);

In [ ]:
/*
    Reference an existing object, but do not take ownership. 
    The C++ side is responsible for managing the object’s lifetime and deallocating it when it is no longer used. 
    Warning: undefined behavior will ensue when the C++ side deletes an object that is still referenced and used by Python.
*/
m.def("get_data", &get_data,return_value_policy::reference);

In [ ]:
/*
    Indicates that the lifetime of the return value is tied to the lifetime of a parent object, namely the implicit this, 
    or self argument of the called method or property. 
    Internally, this policy works just like return_value_policy::reference but additionally 
    applies a keep_alive<0, 1> call policy (described in the next section) that prevents the parent object 
    from being garbage collected as long as the return value is referenced by Python. 
    This is the default policy for property getters created via def_property, def_readwrite, etc.
*/
m.def("get_data", &get_data,return_value_policy::reference_internal);

In [ ]:
/*
    This policy falls back to the policy return_value_policy::take_ownership when the return value is a pointer. 
    Otherwise, it uses return_value_policy::move or return_value_policy::copy for rvalue and lvalue references, respectively. 
    See above for a description of what all of these different policies do. This is the default policy for py::class_- wrapped types.
*/
m.def("get_data", &get_data,return_value_policy::automatic);

In [ ]:
/*
    As above, but use policy return_value_policy::reference when the return value is a pointer. 
    This is the default conversion policy for function arguments when 
    calling Python functions manually from C++ code (i.e. via handle::operator()) and the casters in pybind11/stl.h. 
    You probably won’t need to use this explicitly.
*/
m.def("get_data", &get_data,return_value_policy::automatic_reference);

Return value policies can also be applied to properties:

In [ ]:
class_<MyClass>(m, "MyClass")
    .def_property("data", &MyClass::getData, &MyClass::setData,
                  py::return_value_policy::copy);